## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets//data.csv')
print(data.head(5))
print()
print(data.info())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

### Вывод

Датасет имеет 21525 записи и 12 колонок с данными, состоящих из информации о количестве детей, стаже, возрасте, образовании и его степени клиента, семейное положение, пол, тип занятости, наличие просроченных платежей, среднемесячный доход и цель получения кредита в банке. На первый взгляд бросаются следующие недочеты в данных: отрицательное количество отработанных дней стажа, в уровне образования (среднее и Среднее), по-разному записаны одни и те же цели получения кредита (на покупку автомобиля и на покупку своего автомобиля), наличие пропусков в данных.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data.isna().sum())
print()

data['days_employed'] = data['days_employed'].abs()
data['total_income'] = data['total_income'].abs()
days_employed_mean = data["days_employed"].mean()
total_income_median = data["total_income"].median()

print(days_employed_mean)
print(total_income_median)
print()

data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data['total_income'] = data['total_income'].fillna(total_income_median)
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

66914.72890682236
145017.93753253992

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Пропуски данных были обнаружены в двух столбцах датасета: days_employed - количество отработанных дней, заменили пропущенные значения средним арифметическим (метод mean) и total_income - среднемесячный доход, который заменили на медианное значение по столбцу исходя из того, что неккоректно считать среднее арифметическое для заработной платы. Для получения значений перевел отрицательные значения в положительные. Возможные пропуски связываем с отсутствием дохода и отработанного времени потому что клиент либо безработный, либо только устроился в организацию.

### Замена типа данных

In [3]:
print(data.info())
data = data.astype({"days_employed":'int64', "total_income":'int64'})
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_s

### Вывод

Чтобы лучше анализировать данные необходимо заменить значения с точкой на целочисленные в следующих столбцах - days_employed и total_income, данные в них меняем методом astype, потому что исходные данные имееют тип float64, что говорит о том, что все значения в столбцах - нецелые или целые числа.

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

71
0


### Вывод

Чтобы начать работать с дубликатами в датасете, уравняем регистр во всех столбцах, содержащих текст. Найден 71 дубликат (0,003% от общего числа данных), удалим их методом drop_duplicates. Причиной появления дубликатов в данных может быть наличие несколько кредитов у удного лица, что привело к удвоению данных о нем, содержащихся в датасете.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()

data['lem_purpose'] = data['purpose'].apply(m.lemmatize)
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"


### Вывод

Выделили леммы значений столбца purpose с помощью метода m.lemmatize, сохранили полученные значения в новый столбец датасета lem_purpose методом apply.

### Категоризация данных

In [6]:
def purpose_cat(purpose):
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    else:
        return 'Undefined'
    
data['purpose_category'] = data['lem_purpose'].apply(purpose_cat)
print(data['purpose_category'].unique())
print()
data.head(5)

['недвижимость' 'автомобиль' 'образование' 'свадьба']



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


### Вывод

В результате обработки по столбцу purpose выделены несколько основных целей: недвижимость, автомобиль, образование, свадьба. Эти значения уникальны - проверены методом data['purpose_category'].unique()). Нет ни одного значения Undefined, что говорит о том, что все значения в столбце purpose_category заполнены и мы учили все варианты в фунции purpose_cat.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
res_children_debt = data.query('children >= 0 and children <= 5').pivot_table(index = 'children', values = 'debt')
res_children_debt.style.format("{:.2%}")

#data['has_debt'] = data['debt']
#res_children_debt2 = data.query('children >= 0 and children <= 4').pivot_table(index = 'children', columns ='has_debt', values = 'debt', aggfunc = 'count')
#res_children_debt2['ratio'] = res_children_debt2[1] / res_children_debt2[0] * 100
#res_children_debt2

,debt
children,
0,7.54%
1,9.23%
2,9.45%
3,8.18%
4,9.76%
5,0.00%


### Вывод

Проанадизировав зависимость количества детей от возвратности кредита можно отметить, что количество детей в меньшей степени влияет на возвратность кредита. Так, заемщик с одним, двумя и четырьмя детьми в равной степени возвращают кредит (9,5%). Самая высокая доля возвратности у заемщиков, не имеющих детей и имеющих трех детей - 0,92. Прямой ависимости от количества детей не установлено - выбивается процент возвратности заемщиков с тремя детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
res_familystatus_debt = data.pivot_table(index = 'family_status', values = 'debt')
res_familystatus_debt.style.format("{:.2%}")

,debt
family_status,
в разводе,7.11%
вдовец / вдова,6.57%
гражданский брак,9.35%
женат / замужем,7.55%
не женат / не замужем,9.75%


### Вывод

Анализируя зависимость семейного положения заемщика и возвратность кредита выявлено, что минимальная возвратность у не женатых (не замужних) клиентов и клиентов, состоящих в гражданском браке, хорошая возвратность прослеживается у вдовых клиентов, находящихся в разводе, и состоящих в официальном браке. Зависимости от наличия спутника не установлено, так как большая разница между возвратностью клиента состоящего в гражданском браке и женатым/замужним официально, ровно как и между вдовым и не женатым/не замужней.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data = data.astype({"total_income":'int64'})
print(data['total_income'].quantile([0.1,0.35,0.65,0.9]))

def ti_cat(total_income):
    if total_income <= 78721.10:
        return '1. низкий'
    if total_income > 78721.10 and total_income < 126296.55:
        return '2. ниже среднего'
    if total_income >  126296.55 and total_income < 166930.55:
        return '3. средний'
    if total_income > 166930.55 and total_income < 269797.40:
        return '4. выше среднего'
    if total_income >= 269797.40:
        return '5. высокий'
    
data['ti_category'] = data['total_income'].apply(ti_cat)

res_totalincome_debt = data.pivot_table(index = 'ti_category', values = 'debt')
res_totalincome_debt.style.format("{:.2%}")

0.10     78721.10
0.35    126296.55
0.65    166930.55
0.90    269797.40
Name: total_income, dtype: float64


,debt
ti_category,
1. низкий,7.32%
2. ниже среднего,8.65%
3. средний,8.48%
4. выше среднего,7.89%
5. высокий,7.04%


### Вывод

Клиенты банка с высоким уровнем дохода чаще возвращают кредиты в срок, лидерами по не возврату кредита являются клиенты с доходом ниже среднего и среднего. Прямой завичимости от уровня дохода не установлено.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
res_purpose_debt = data.pivot_table(index = 'purpose_category', values = 'debt')
res_purpose_debt.style.format("{:.2%}")

,debt
purpose_category,
автомобиль,9.36%
недвижимость,7.23%
образование,9.22%
свадьба,8.00%


### Вывод

Лидерами по невозвратности кредита являются клиенты, которые взяли кредит на автомобиль или образование. Возвратные же цели присущи для жилищных кредитов и взятых для проведения свадьбы. Полагаем, что это связано с риском разбить автомобиль и не закончить образование. Возвратность жилищных кредитов сопровождается наличием мер поддержки - материнский капитал, возврат налогов с покупки жилья и с уплаченных процентов, а возвратность кредитов на свадьбу обусловлена денежными подарками, за счет которых и погашен кредит.

### Шаг 4. Общий вывод

Анализируя различные зависимости от не пришлось выделить прямой от количества детей, семейного положения и уровня дохода. Полагаю, что эти данные необходимы для построения скоринговых моделей надежности заемщика, рассматриваемого как отдельную единицу по совокупности признаков. Что касаемо целей кредитования, выявлено, что цель кредита связана с возвратностью напрямуюв зависимости от дальнейшего возможных поступлений денежных средств, которые могут быть напрвлены на погашение кредита.  